In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'sampleSubmission.csv', 'train.csv']


# 1. Data EDA

* ## 1.1 Load Data

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [4]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [5]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


train과 test 데이터 컬럼을 먼저 살펴보자. casual, registered, count가 test 데이터에는 없다. 우리는 test data를 predict할 것이다

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
datetime      10886 non-null object
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.6+ KB


train데이터를 살펴본다. 

null data가 없는지 본다. 

type에 datetime은 object로 되어있는데 쪼개 주어야 한다.

지금은 작은데이터니까 int64, float64로 두지만 큰 데이터를 할 때에는 메모리 조절을 데이터를 보며 잘 해줘야 램이 안터진다. 

In [7]:
train.shape

(10886, 12)

train 데이터의 컬럼은 12개, 데이터는 10886개임을 확인.

In [9]:
train.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

이런 방법으로도 null의 합이 0 (null data가 없음) 을 확인할 수 있다.

## 1.2 데이터 가공

> ### 1.2.1 datetime 쪼개주기

datetime 은 object형식인데, 숫자로 바꿔서 학습에 넣어줄것이다.

In [25]:
train = pd.read_csv("../input/train.csv", parse_dates = ["datetime"])
test = pd.read_csv("../input/test.csv", parse_dates = ["datetime"])

In [26]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
datetime      10886 non-null datetime64[ns]
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.6 KB


datetime의 형식이 바뀌었음을 볼 수 있다. 

datetime에서 연도, 시간, 요일을 가져올 것이다. 

이 데이터에서 연도는 2011, 2012 두가지가 있기 때문에 가져오는 것이고,

월은 season에서 반영이 되어있기때문에 가져오지 않고,

일은 train과 test데이터에서 날짜가 나뉘어 있어서 가져오지 않는 것이 훨씬 좋다.

분, 초는 0이어서 가져오지 않는다.

In [28]:
train["year"] = train["datetime"].dt.year
train["hour"] = train["datetime"].dt.hour
train["dayofweek"] = train["datetime"].dt.dayofweek

test["year"] = test["datetime"].dt.year
test["hour"] = test["datetime"].dt.hour
test["dayofweek"] = test["datetime"].dt.dayofweek

In [29]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,hour,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,2,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,3,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,4,5


In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 15 columns):
datetime      10886 non-null datetime64[ns]
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
year          10886 non-null int64
hour          10886 non-null int64
dayofweek     10886 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(11)
memory usage: 1.2 MB


In [31]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 12 columns):
datetime      6493 non-null datetime64[ns]
season        6493 non-null int64
holiday       6493 non-null int64
workingday    6493 non-null int64
weather       6493 non-null int64
temp          6493 non-null float64
atemp         6493 non-null float64
humidity      6493 non-null int64
windspeed     6493 non-null float64
year          6493 non-null int64
hour          6493 non-null int64
dayofweek     6493 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 608.8 KB


In [32]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,hour,dayofweek
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,0,3
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,3
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,2,3
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,3,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,4,3


학습시킬 레이블은 y라는 변수로 따로 빼준다.

In [33]:
y_train = train["count"]

In [34]:
y_train.head()

0    16
1    40
2    32
3    13
4     1
Name: count, dtype: int64

y의 분포가 skew 되어있기 때문에 로그화를 해준다. 대부분의 데이터는 로그화로 충분하게 스케일된다.

In [35]:
y_train = np.log1p(y_train)

학습시킬 feature에서 없앨 것을 없앤다. 

datetime은 쪼갰으니까 없애고, 

windspeed는 큰 영향을 주지 않는다고 판단해서 없애고

casual과 registered는 우선 없애고 다음 커널에서 활용하기 위해 없애고

count는 우리가 예측할 값으로 빼주었으므로 지운다. 

1은 axis방향이고, inplace=True는 지금바로 드랍할것인지를 의미한다.

In [36]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,hour,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,2,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,3,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,4,5


In [37]:
train.drop(["datetime", "windspeed",  "casual", "registered", "count"], 1, inplace=True)

In [38]:
train.head()

,season,holiday,workingday,weather,temp,atemp,humidity,year,hour,dayofweek
0,1,0,0,1,9.84,14.395,81,2011,0,5
1,1,0,0,1,9.02,13.635,80,2011,1,5
2,1,0,0,1,9.02,13.635,80,2011,2,5
3,1,0,0,1,9.84,14.395,75,2011,3,5
4,1,0,0,1,9.84,14.395,75,2011,4,5


 test도 마찬가지로 지워준다.
 
 얘는 원래 casual, registered, count는 없었으니 두개만 지워준다.

In [39]:
test.drop(["datetime", "windspeed"], 1, inplace=True)

In [41]:
test.head()

,season,holiday,workingday,weather,temp,atemp,humidity,year,hour,dayofweek
0,1,0,1,1,10.66,11.365,56,2011,0,3
1,1,0,1,1,10.66,13.635,56,2011,1,3
2,1,0,1,1,10.66,13.635,56,2011,2,3
3,1,0,1,1,10.66,12.880,56,2011,3,3
4,1,0,1,1,10.66,12.880,56,2011,4,3


# 2. 모델링

랜덤포리스트를 베이스라인으로 만들어보자. RandomForestRegressor를 가져온다. 대부분의 모델들이 Classifier와 Regressor로 두개씩 있다.

In [42]:
from sklearn.ensemble import RandomForestRegressor

estimator는 많다고 좋은것은 아니다.  적당한게 좋다.

In [43]:
rf = RandomForestRegressor(n_estimators=100)

In [44]:
rf.fit(train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [62]:
preds = rf.predict(test)

제출할 파일을 만든다.

In [63]:
submission=pd.read_csv("../input/sampleSubmission.csv")

count 컬럼에 데이터를 넣어준다.

In [64]:
submission["count"] = preds 

In [65]:
submission.head()

,datetime,count
0,2011-01-20 00:00:00,2.527208
1,2011-01-20 01:00:00,1.791576
2,2011-01-20 02:00:00,1.339690
3,2011-01-20 03:00:00,1.563833
4,2011-01-20 04:00:00,1.328329


count값이 이상한걸 확인할 수 있다. 아까 우리가 로그화를 해서 그렇다. 다시 돌려주자.

In [68]:
# preds = np.expm1(preds)
# submission["count"] = preds 

In [66]:
submission["count"] = np.expm1(preds)

In [67]:
submission.head()

,datetime,count
0,2011-01-20 00:00:00,11.518502
1,2011-01-20 01:00:00,4.998900
2,2011-01-20 02:00:00,2.817859
3,2011-01-20 03:00:00,3.777095
4,2011-01-20 04:00:00,2.774730


count값이 정상적으로 돌아왔다. 제출할수 있다!
csv파일로 만들어주자.

In [69]:
submission.to_csv("allrf.csv", index=False)

# 3. Results

public score 0.39513이 나왔다. 이 대회는 이미 4년 전에 끝난 대회여서 아쉽게도 등수 확인은 못 하지만, 예전 리더보드 기준으로 3251팀 중 185등, 약 상위 5.69%의 성적임을 확인할 수 있다. 

물론 아직 진행되고 있는 대회는 아니지만, 데이터 관찰과 분석, 그리고 랜덤포리스트 베이스라인 모델만으로 거둔 것에 의미가 있다. 

다음 커널에서는 casual과 registered 컬럼을 활용해서 결과를 조금 더 올려보겠다.